# LongSlit-02:   BIAS correction 

## Overview
1. Build a list of BIAS image files
2. Display and statistics
3. Overscan correction and trimming
3. Combine to obtain master BIAS

<pre>
Máster en Astrofísica UCM  -- Técnicas Experimentales en Astrofísica  
Jaime Zamorano, Nicolás Cardiel and Sergio Pascual

Version 1.0 2021/02/12  
</pre>

Note that the ``astropy`` package should be installed. In this sense, have a look to the
astropy installation description: https://docs.astropy.org/en/stable/install.html.

In [ ]:
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import ccdproc
from ccdproc import CCDData, Combiner, subtract_overscan

In [ ]:
plt.style.use('./tea.mplstyle')    # Some parameters for nicer graphs

### Working with images in a directory
We will use the files of the first night of observations at NOT (Nordic Optical Telescope) 2008 that can be downloaded from   ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/N1/
or 
http://guaix.fis.ucm.es/~jaz/master_TEA/observaciones_NOT_2008/N1/

``directory`` should point to the directory with the downloaded images

In [ ]:
directory='/Users/jzamorano/Desktop/INT_1997/N1/'

Let\'s create a list containing all the BIAS files in that directory 

In [ ]:
from ccdproc import ImageFileCollection
from ccdproc.utils.sample_directory import sample_directory_with_files
ic_all = ImageFileCollection(directory, keywords='*')
print(ic_all.summary.colnames)

In [ ]:
ic_all.summary['file', 'imagetyp', 'exptime', 'naxis1', 'naxis2']

In [ ]:
keys = ['file','OBJECT' , 'EXPTIME' , 'OBSTYPE' , 'CCDSTEMP', 'CCDATEMP']
ic1 = ImageFileCollection(directory, keywords=keys) # only keep track of keys
ic1.summary.colnames

In [ ]:
bias_images = ic1.files_filtered(regex_match=True,imagetyp='bias|light')
print(bias_images)

In [ ]:
image_bias, filename_bias = [], []
for file in bias_images:
    print(file)
    image = CCDData.read(directory+file, unit="adu")
    image_bias.append(image)
    filename_bias.append(file)

In [ ]:
# auxiliary function to display a rectangle and compute mean value within it
def draw_rectangle(ax, image_data, x1, x2, y1, y2, color, text=False):
    ax.plot((x1, x1), (y1, y2), color, lw=1)
    ax.plot((x2, x2), (y1, y2), color, lw=1)
    ax.plot((x1, x2), (y1, y1), color, lw=1)
    ax.plot((x1, x2), (y2, y2), color, lw=1)
    if text:
        media = image_data[y1:y2,x1:x2].mean()
        std   = image_data[y1:y2,x1:x2].std()
        ax.text((x1+x2)/2, y1+(y2-y1)/8, str(int(media)), 
                ha='center', va='center', color=color, fontsize=15)        
        ax.text((x1+x2)/2, y2-(y2-y1)/8, str(round(std,1)), 
                ha='center', va='top', color=color, fontsize=15)
    return media, std

In [ ]:
mean_bias, std_bias, expo_bias = [], [], []
fig, ax = plt.subplots(ncols=1, nrows=len(bias_images), figsize=(14, 20))
n = 0
for i in range(len(bias_images)):
    img = ax[n].imshow(image_bias[i], cmap='gray', origin='lower',vmin=685,vmax=700)
    ax[n].set_xlabel('X axis')
    ax[n].set_ylabel('Y axis')
    ax[n].text(100,200,filename_bias[i],color='w',fontsize=20)
    divider = make_axes_locatable(ax[n])
    mean,std = draw_rectangle(ax[n], image_bias[i].data , 400, 600, 50, 200, color='w',text=True)
    mean_bias.append(mean)
    std_bias.append(std)
    expo_bias.append(image_bias[i].header['EXPTIME'])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax)
    n = n+1

In [ ]:
bias_images = np.delete(bias_images,2)

### Checking dark current

In [ ]:
dark_images = ic1.files_filtered(regex_match=True,object='dark')
print(dark_images)

In [ ]:
image_dark , filename_dark , expo_dark = [], [], []
for file in dark_images:
    print(directory+file)
    image = CCDData.read(directory+file, unit="adu")
    image_dark.append(image)
    filename_dark.append(file)    
    expo_dark.append(image.header['EXPTIME'])

In [ ]:
mean_dark, std_dark = [], []
fig, ax = plt.subplots(ncols=1, nrows=len(dark_images), figsize=(14, 6))
n = 0
for i in range(len(dark_images)):
    img = ax[n].imshow(image_dark[i], cmap='gray', origin='lower',vmin=650,vmax=750)
    ax[n].set_xlabel('X axis')
    ax[n].set_ylabel('Y axis')
    ax[n].text(100,200,filename_dark[i],color='w',fontsize=20)
    ax[n].text(900,200,str(expo_dark[i])+' s',color='w',fontsize=20)
    divider = make_axes_locatable(ax[n])
    draw_rectangle(ax[n], image_dark[i].data , 700, 850, 50, 200, color='w',text=True)
    mean,std = draw_rectangle(ax[n], image_dark[i].data , 80, 160, 10, 100, color='w',text=True)
    mean_dark.append(mean)
    std_dark.append(std)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax)
    n = n+1

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(9, 4))
ax.plot(expo_bias,mean_bias,'bo')
ax.plot(expo_dark,mean_dark,'ro')
ax.set_xlabel('texpo [s]')
ax.set_ylabel('Mean [counts]')

## Substract overscan

Before creating a master_BIAS we should correct from overscan.  

In [ ]:
image_bias_no_scan = []
for i in range(len(bias_images)):
    no_scan = subtract_overscan(image_bias[i], overscan=image_bias[i][:, 0:50])
    image_bias_no_scan.append(no_scan)

## superBIAS

In [ ]:
combiner = Combiner(image_bias_no_scan)

In [ ]:
master_BIAS = combiner.median_combine()

We trim the resulting master_BIAS using Python simple operation on arrays.

In [ ]:
master_BIAS_N1 = master_BIAS[:,50:-50]
print(master_BIAS.shape, master_BIAS_N1.shape)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(14, 9))
img = ax[0].imshow(image_bias[0], cmap='gray', origin='lower',vmin=685,vmax=700)
ax[0].text(100,200,filename_bias[0],color='k',fontsize=20)
draw_rectangle(ax[0], image_bias[0].data , 400, 600, 30, 170, color='w',text=True)
img = ax[1].imshow(image_bias[-1], cmap='gray', origin='lower',vmin=685,vmax=700)
ax[1].text(100,200,filename_bias[-1],color='w',fontsize=20)
draw_rectangle(ax[1], image_bias[-1].data , 400, 600, 30, 170, color='w',text=True)
img = ax[2].imshow(master_BIAS_N1, cmap='gray', origin='lower',vmin=0,vmax=5)
ax[2].text(100,200,'master_BIAS_N1',color='w',fontsize=20)
for n in range(6):
    draw_rectangle(ax[2], master_BIAS.data , 50+n*150, 200+n*150, 30, 170, color='w',text=True)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,4))
step = 2
bins = np.arange(670,720,step)
h_single = ax.hist(image_bias[0].data.flatten(), bins=bins, density=True, 
             label="single BIAS", alpha=0.4);
# Adding mean value of sinle BIAS to compare histograms
h_master = ax.hist(master_BIAS_N1.data.flatten()+693, bins=bins, density=True, 
             label="master_BIAS_N1", alpha=0.4);
ax.legend()
ax.grid()
#print(h_single)

# To fit a gaussian
from scipy.optimize import curve_fit      
xx = np.arange(670,720,0.2)
# Gaussian function definition
def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))
# Transforming from bins edges to bins' centers
x = np.add(h_single[1][:-1],step/2.)
# values obtained from histogram
y = h_single[0]
# Fitting  (with some initial guesses)
popt,pcov = curve_fit(gaus,x,y,p0=[1,690,10])
# From results we can get center and width 
sigma = popt[2]
# Plotting the fit 
y_new = gaus(xx,*popt)
plt.plot(xx,y_new,lw=2,label="norm sigma = "+str(round(sigma,1)))
# again for the master_BIAS
y = h_master[0]
popt,pcov = curve_fit(gaus,x,y,p0=[1,690,10])
sigma = popt[2]
y_new = gaus(xx,*popt)
plt.plot(xx,y_new,lw=2,label="norm sigma = "+str(round(sigma,1)))

ax.legend()

### Checking BIAS correction
#21 45246.fits 600.0     U1256+2722 PA=315 171

In [ ]:
image_science = CCDData.read(directory+'45246.fits', unit="adu")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(14,6))
img = ax.imshow(image_science, cmap='viridis', origin='lower',vmin=680,vmax=740)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.text(150,200,image_science.header['object'],color='w',fontsize=20)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax)
draw_rectangle(ax, image_science.data , 720, 820, 110, 210, color='w',text=True)
draw_rectangle(ax, image_science.data , 0, 50, 40, 240, color='w',text=True)


In [ ]:
    science_no_scan = subtract_overscan(image_science, overscan=image_science[:, 0:50])

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(14,6))
img = ax.imshow(science_no_scan, cmap='viridis', origin='lower',vmin=0,vmax=60)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.text(150,200,science_no_scan.header['object'],color='w',fontsize=20)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax)
draw_rectangle(ax, science_no_scan.data , 720, 820, 110, 210, color='w',text=True)
draw_rectangle(ax, science_no_scan.data , 0, 50, 40, 240, color='w',text=True)


In [ ]:
t_science_image = ccdproc.trim_image(science_no_scan[:,50:-50])

In [ ]:
bias_subtracted_science = ccdproc.subtract_bias(t_science_image, master_BIAS_N1)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(14,6))
img = ax.imshow(bias_subtracted_science, cmap='viridis', origin='lower',vmin=0,vmax=50)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.text(150,200,bias_subtracted_science.header['object'],color='w',fontsize=20)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax)
draw_rectangle(ax, bias_subtracted_science.data , 670, 770, 110, 210, color='w',text=True)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(14, 6))
img = ax[0].imshow(science_no_scan, cmap='viridis', origin='lower',vmin=0,vmax=40)
draw_rectangle(ax[0], science_no_scan.data , 720, 820, 110, 210, color='w',text=True)
draw_rectangle(ax[0], science_no_scan.data , 350, 450, 150, 210, color='w',text=True)

img = ax[1].imshow(bias_subtracted_science, cmap='viridis', origin='lower',vmin=0,vmax=40)
draw_rectangle(ax[1], bias_subtracted_science.data, 670, 770, 110, 210, color='w',text=True)
draw_rectangle(ax[1], bias_subtracted_science.data, 300, 400, 150, 210, color='w',text=True)


In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,4))
bins=np.arange(-30,30,2)
ax.hist(science_no_scan.data.flatten(), bins=bins, align='mid', density=True, 
             label="original", alpha=0.4);
ax.hist(bias_subtracted_science.data.flatten(), bins=bins, align='mid', density=True, 
             label="BIAS subtracted", alpha=0.4);
ax.legend()